In [1]:
%matplotlib notebook
import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
import unittest
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from deepART import dataset, torch
from deepART.torch import ProbART
from deepART import probART
import timeit
import torch
from concurrent.futures import ThreadPoolExecutor

/usr/local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
ndata = 100
ndim = 27000*4

In [4]:
# torch tensor
data = torch.randint(low=0, high=1000, size=(ndata,ndim))/1000

In [5]:
#numpy array
data = np.random.rand(ndata,ndim)

In [6]:
convergence = 5
n=ndim
neurons = 50
rho = 0.5
beta = 0.5
alpha = 0.1
main_executor = ThreadPoolExecutor(max_workers=32)

network = probART.ProbART( n=n, m=neurons, rho=rho, alpha=alpha, beta=beta, comp=False, fastEncode=1)
#network = ProbART( n=n, m=neurons, rho=rho, alpha=alpha, beta=beta, comp=False, fastEncode=1)
#network.cuda()
timer = timeit.Timer()
print("Start testing...")


#convergence tracking variables
prev_active = 0
converge = 0
results = []
start_time = timer.timer()
while True:
    for I in data:
        Z, k = network.fit_predict(I,threading=main_executor)
        #Z, k = network.fit_predict(I)

    if (prev_active < network.active):
        prev_active = network.active
        converge +=1
        continue
    else:
        converge +=1
        if converge > convergence:
            print("Total Neurons Learned: {}\n\n".format(network.active))
            break
#183.11420114199996
#163.720597932
print("Training completed...Took:{}".format((timer.timer() - start_time)))

Start testing...
Total Neurons Learned: 50


Training completed...Took:41.415720412999974


In [7]:
#numpy
avg = np.mean([4.8344, 4.98877, 4.8410]) #10000
avg = np.mean([93.38413996400027, 92.60151783900028, 92.59009739699968]) #27000*4
avg


92.85858506666675

In [8]:
#numpy - multithread 20 workers
avg = np.mean([3.8488126999999395, 3.6574866490000204, 3.7809610430000475]) #10000
avg = np.mean([29.858683285, 28.780577772, 28.80568363699996]) #27000*4
avg


29.148314897999985

In [9]:
#pyTorch - no cuda
avg = np.mean([3.8709, 3.926874, 3.81118])
avg

3.8696513333333336

In [10]:
#pyTorch - cuda
avg = np.mean([24.2966, 23.8491, 24.16625])
avg = np.mean([54.47145563999993, 53.82952262899971,53.824207165999724]) #27000*4
avg

54.04172847833312